In [3]:
import pandas as pd

dataset=pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
dataset.label.value_counts()

label
0    3000
1    2242
Name: count, dtype: int64

In [7]:
dataset.shape

(5242, 3)

In [9]:
for index, tweet in enumerate (dataset ["tweet"] [10:15]):
    print (index+1,"-", tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [11]:
import re
def clean_text (text):
    text = re.sub(r'[^a-zA-Z\']',' ', text)
    text = re.sub(r'[^|x00-|x7F]+',' ',text)
    text = text.lower()
    return text

In [13]:
dataset['clean_text'] = dataset.tweet.apply(lambda x:clean_text(x))

In [15]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is so...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t use ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before they lea...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champions clev...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr


In [17]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [18]:
stop = stopwords.words('english')

In [39]:
def gen_freq(text):
    word_list=[]
    for tw_words in text.split():
        word_list.extend(tw_words)
    word_freq=pd.Series(word_list).value_counts()
    word_freq=word_freq.drop(stop,errors='ignore')
    return word_freq

In [41]:
def any_neg(words):
    for word in words:
        if word in ['n','no','non','not'] or re.search(r"\wn't",word):
            return 1
        else:
            return 0

In [47]:
def any_rare(words,rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0

In [49]:
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who','where']:
            return 1
        else:
            return 0

In [51]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:] 
dataset ['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
dataset ['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset ['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset['any_rare'] =dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
dataset['char_count'] =dataset.clean_text.apply(lambda x:len (x))

In [55]:
dataset.head(10)

,id,label,tweet,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is so...,18,0,0,0,97
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t use ...,21,0,0,0,114
2,3,0,bihday your majesty,bihday your majesty,3,0,0,0,20
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur,12,0,0,0,47
4,5,0,factsguide: society now #motivation,factsguide society now motivation,4,0,0,0,34
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before they lea...,18,0,0,0,107
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user user use...,11,0,0,0,64
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...,21,0,0,0,124
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champions clev...,10,0,0,0,71
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr,10,0,0,0,39


In [61]:
from sklearn.model_selection import train_test_split
X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset. label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0)

In [63]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()

model=model.fit(X_train,y_train)
pred=model.predict(X_test)

In [65]:
model.predict(X_test[5:10])

array([0, 1, 1, 1, 1])

In [67]:
from sklearn.metrics import accuracy_score

print("Accuracy: ",accuracy_score(y_test,pred)*100,"%")

Accuracy:  42.89799809342231 %


In [69]:
from sklearn.ensemble import RandomForestClassifier

clf_rf=RandomForestClassifier()

clf_rf.fit(X_train,y_train)
rf_pred=clf_rf.predict(X_test).astype(int)

In [71]:
from sklearn.metrics import classification_report, confusion_matrix, \
accuracy_score
print (confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))
print ("Accuracy:", accuracy_score(y_test, rf_pred) )

[[440 159]
 [236 214]]
              precision    recall  f1-score   support

           0       0.65      0.73      0.69       599
           1       0.57      0.48      0.52       450

    accuracy                           0.62      1049
   macro avg       0.61      0.61      0.61      1049
weighted avg       0.62      0.62      0.62      1049

Accuracy: 0.6234509056244042


In [73]:
from sklearn. linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [77]:
y_pred = logreg.predict(X_test)

In [79]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.56      0.59       599
           1       0.49      0.58      0.53       450

    accuracy                           0.57      1049
   macro avg       0.57      0.57      0.56      1049
weighted avg       0.58      0.57      0.57      1049

